In [ ]:
# ============================================================
# CONFIGURACIÓN E IMPORTS
# ============================================================

import re
import pandas as pd
import os
from glob import glob
from pathlib import Path
import sys

# Agregar la raíz del proyecto al path
ROOT = Path().resolve().parent
sys.path.insert(0, str(ROOT))

# Importar configuración centralizada
from config import PATHS, CONFIG

print("✓ Configuración cargada correctamente")
print(f"Raíz del proyecto: {PATHS.ROOT}")

In [ ]:
# ============= CONFIGURACIÓN USANDO CONFIG.PY =============

# Usar rutas centralizadas
CARPETA_PROCESADOS = PATHS.UNCLEANED
CARPETA_LIMPIA = PATHS.PROCESSED

# Usar configuración centralizada de limpieza
MANTENER_HASHTAGS = CONFIG.MANTENER_HASHTAGS
MANTENER_MENCIONES = CONFIG.MANTENER_MENCIONES
MANTENER_URLS = CONFIG.MANTENER_URLS
LONGITUD_MINIMA_TEXTO = CONFIG.LONGITUD_MINIMA_TEXTO
PALABRAS_MINIMAS = CONFIG.PALABRAS_MINIMAS

print("="*60)
print("CONFIGURACIÓN ACTIVA:")
print("="*60)
print(f"CSVs individuales: {CARPETA_PROCESADOS}")
print(f"Datos limpios: {CARPETA_LIMPIA}")
print(f"Mantener hashtags: {MANTENER_HASHTAGS}")
print(f"Mantener menciones: {MANTENER_MENCIONES}")
print(f"Mantener URLs: {MANTENER_URLS}")
print(f"Longitud mínima: {LONGITUD_MINIMA_TEXTO} caracteres")
print(f"Palabras mínimas: {PALABRAS_MINIMAS}")
print("="*60)

In [3]:
def consolidar_csvs(carpeta):
    """
    Consolida todos los archivos CSV de una carpeta en un solo DataFrame
    
    Args:
        carpeta: Ruta de la carpeta con los CSVs
    
    Returns:
        DataFrame consolidado
    """
    print(f"\n{'='*80}")
    print("CONSOLIDANDO ARCHIVOS CSV")
    print(f"{'='*80}")
    
    # Buscar todos los archivos CSV
    patron = os.path.join(carpeta, "*.csv")
    archivos_csv = glob(patron)
    
    if not archivos_csv:
        print(f"⚠️  No se encontraron archivos CSV en '{carpeta}/'")
        return None
    
    print(f"📁 Archivos encontrados: {len(archivos_csv)}")
    
    dataframes = []
    
    for archivo in sorted(archivos_csv):
        nombre = os.path.basename(archivo)
        try:
            df = pd.read_csv(archivo, encoding='utf-8-sig')
            dataframes.append(df)
            print(f"  ✓ {nombre}: {len(df)} registros")
        except Exception as e:
            print(f"Error en {nombre}: {e}")
            continue
    
    if not dataframes:
        print("No se pudieron leer archivos CSV")
        return None
    
    # Consolidar todos los DataFrames
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    
    # Eliminar duplicados por id_publicacion
    if 'id_publicacion' in df_consolidado.columns:
        antes = len(df_consolidado)
        df_consolidado = df_consolidado.drop_duplicates(subset=['id_publicacion'])
        duplicados = antes - len(df_consolidado)
        if duplicados > 0:
            print(f"\n⚠️  Se eliminaron {duplicados} registros duplicados")
    
    print(f"\n✓ Total de registros consolidados: {len(df_consolidado)}")
    
    return df_consolidado

In [4]:
def limpiar_texto(texto, mantener_hashtags=False, mantener_menciones=False, mantener_urls=False):
    """
    Limpia y normaliza el texto de publicaciones
    
    Args:
        texto: Texto a limpiar
        mantener_hashtags: Si True, mantiene los hashtags
        mantener_menciones: Si True, mantiene las menciones
        mantener_urls: Si True, mantiene las URLs

    Returns:
        Texto limpio
    """
    if pd.isna(texto) or texto == "":
        return ""

    # Convertir a string (mantener caso por ahora para eliminar patrones específicos)
    texto = str(texto)

    # Eliminar contenido entre directional isolates (LRI/RLI/FSI ... PDI)
    # Ej: \u2066...\u2069 (usado en algunas exportaciones para envolver nombres)
    texto = re.sub(r'[\u2066\u2067\u2068].*?\u2069', '', texto)
    # Eliminar contenido entre embedding/override markers y su terminador (U+202C)
    texto = re.sub(r'[\u202A-\u202E].*?\u202C', '', texto)

    # Quitar cualquier carácter directional aislado remanente
    for ch in ['\u2066', '\u2067', '\u2068', '\u2069', '\u202a', '\u202b', '\u202c', '\u202d', '\u202e']:
        texto = texto.replace(ch, '')

    # Eliminar marcadores de multimedia y mensajes eliminados (varias variantes, case-insensitive)
    texto = re.sub(r'<\s*multimedia\s+omitido\s*>', '', texto, flags=re.I)
    texto = re.sub(r'eliminaste\s+este\s+mensaje\.?', '', texto, flags=re.I)

    # Si no queremos mantener menciones, eliminar las menciones simples y también
    # menciones de varias palabras (por ejemplo nombres completos) precedidas por @
    if not mantener_menciones:
        # menciones simples tipo @usuario
        texto = re.sub(r'@\w+', '', texto)
        # menciones multi-palabra que pueden aparecer como '@ Nombre Apellido' o ' @⁨Nombre Apellido⁩'
        texto = re.sub(r'@\s*[A-ZÁÉÍÓÚÑÜ][\w\-\u00C0-\u017F\.\s]{0,120}?\b', '', texto)
        # También eliminar cualquier '@' suelto que pueda quedar (p.ej. cuando el nombre estaba envuelto en isolates)
        texto = re.sub(r'@\s*', '', texto)

    # También eliminar etiquetas o nombres que terminan en 'IS' (ej: 'Miguel IS') si quedan
    texto = re.sub(r'\b[\w\-\u00C0-\u017F]+\s+IS\b', '', texto, flags=re.I)
    # Eliminar la etiqueta 'IS' sola si queda
    texto = re.sub(r'\bIS\b', '', texto)

    # Ahora convertir a minúsculas para el resto del procesamiento
    texto = texto.lower()

    # Eliminar URLs (si no se quieren mantener)
    if not mantener_urls:
        texto = re.sub(r'http\S+|www\S+|https\S+', '', texto)

    # Eliminar hashtags (si no se quieren mantener)
    if not mantener_hashtags:
        texto = re.sub(r'#\w+', '', texto)

    # Eliminar emojis y caracteres especiales (manteniendo signos de puntuación básicos)
    texto = re.sub(r'[^\w\s.,!?¿¡áéíóúñü]', '', texto)

    # Normalizar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()

    # Eliminar puntos y comas al inicio/final
    texto = texto.strip('.,')

    return texto

In [5]:
def preprocesar_datos():
    """
    Proceso completo de consolidación y limpieza de datos
    """
    print("="*80)
    print("PREPROCESAMIENTO DE DATOS")
    print("="*80)
    
    # Crear carpeta de datos limpios si no existe
    if not os.path.exists(CARPETA_LIMPIA):
        os.makedirs(CARPETA_LIMPIA)
        print(f"✓ Carpeta '{CARPETA_LIMPIA}/' creada")
    
    # Verificar que existe la carpeta de procesados
    if not os.path.exists(CARPETA_PROCESADOS):
        print(f"\nError: La carpeta '{CARPETA_PROCESADOS}/' no existe")
        return None
    
    # PASO 1: Consolidar todos los CSVs
    df_consolidado = consolidar_csvs(CARPETA_PROCESADOS)
    
    if df_consolidado is None or df_consolidado.empty:
        print("No se pudo consolidar los datos")
        return None
    
    # PASO 1.5: Aplicar criterios de exclusión
    print(f"\n{'='*80}")
    print("APLICANDO CRITERIOS DE EXCLUSIÓN")
    print(f"{'='*80}")
    
    # Participantes a excluir (< 10 publicaciones o datos incompletos)
    PARTICIPANTES_EXCLUIDOS = ['EST005', 'EST006', 'EST022']
    
    n_antes = len(df_consolidado)
    df_consolidado = df_consolidado[~df_consolidado['id_participante'].isin(PARTICIPANTES_EXCLUIDOS)]
    n_excluidos = n_antes - len(df_consolidado)
    
    if n_excluidos > 0:
        print(f"✓ Excluidos {n_excluidos} registros de {len(PARTICIPANTES_EXCLUIDOS)} participantes:")
        for pid in PARTICIPANTES_EXCLUIDOS:
            print(f"  - {pid} (< 10 publicaciones o datos incompletos)")
    
    print(f"✓ Registros restantes: {len(df_consolidado)}")
    
    # Verificar columnas requeridas
    columnas_requeridas = ['id_participante', 'id_publicacion', 'fecha_publicacion', 'texto_publicacion']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df_consolidado.columns]
    
    if columnas_faltantes:
        print(f"\nError: Columnas faltantes: {columnas_faltantes}")
        return None
    
    print(f"\n{'='*80}")
    print("LIMPIANDO TEXTO")
    print(f"{'='*80}")
    
    # PASO 2: Limpiar texto
    print(f"Configuración de limpieza:")
    print(f"  - Mantener hashtags: {MANTENER_HASHTAGS}")
    print(f"  - Mantener menciones: {MANTENER_MENCIONES}")
    print(f"  - Mantener URLs: {MANTENER_URLS}")
    print(f"  - Longitud mínima: {LONGITUD_MINIMA_TEXTO} caracteres")
    print(f"  - Palabras mínimas: {PALABRAS_MINIMAS}")
    
    # Aplicar limpieza
    df_consolidado['texto_limpio'] = df_consolidado['texto_publicacion'].apply(
        lambda x: limpiar_texto(
            x, 
            mantener_hashtags=MANTENER_HASHTAGS,
            mantener_menciones=MANTENER_MENCIONES,
            mantener_urls=MANTENER_URLS
        )
    )
    
    # Calcular estadísticas de longitud
    df_consolidado['longitud_original'] = df_consolidado['texto_publicacion'].str.len()
    df_consolidado['longitud_limpia'] = df_consolidado['texto_limpio'].str.len()
    df_consolidado['num_palabras'] = df_consolidado['texto_limpio'].str.split().str.len()
    
    print(f"\n✓ Texto limpiado exitosamente")
    
    # PASO 3: Filtrar registros
    print(f"\n{'='*80}")
    print("FILTRANDO REGISTROS")
    print(f"{'='*80}")
    
    total_original = len(df_consolidado)
    
    # Eliminar registros completamente vacíos
    df_consolidado_limpio = df_consolidado[
        (df_consolidado['texto_limpio'].notna()) & 
        (df_consolidado['texto_limpio'] != "")
    ].copy()
    
    vacios_eliminados = total_original - len(df_consolidado_limpio)
    print(f"  - Registros vacíos eliminados: {vacios_eliminados}")
    
    # Filtrar por longitud mínima de texto
    df_texto_valido = df_consolidado_limpio[
        (df_consolidado_limpio['longitud_limpia'] >= LONGITUD_MINIMA_TEXTO) &
        (df_consolidado_limpio['num_palabras'] >= PALABRAS_MINIMAS)
    ].copy()
    
    cortos_eliminados = len(df_consolidado_limpio) - len(df_texto_valido)
    print(f"  - Registros muy cortos eliminados: {cortos_eliminados}")
    
    print(f"\n✓ Total de registros finales: {len(df_texto_valido)}")
    print(f"  ({(len(df_texto_valido)/total_original*100):.1f}% del total original)")
    
    # PASO 4: Guardar archivos
    print(f"\n{'='*80}")
    print("GUARDANDO ARCHIVOS")
    print(f"{'='*80}")
    
    # Archivo 1: Consolidado sin procesar
    archivo_consolidado = os.path.join(CARPETA_LIMPIA, 'publicaciones_consolidado.csv')
    df_consolidado.to_csv(archivo_consolidado, index=False, encoding='utf-8-sig')
    print(f"  ✓ {archivo_consolidado}")
    print(f"    Total: {len(df_consolidado)} registros")
    
    # Archivo 2: Consolidado con texto limpio (todos los registros)
    archivo_completo = os.path.join(CARPETA_LIMPIA, 'publicaciones_completas.csv')
    df_consolidado_limpio.to_csv(archivo_completo, index=False, encoding='utf-8-sig')
    print(f"\n  ✓ {archivo_completo}")
    print(f"    Total: {len(df_consolidado_limpio)} registros")
    print(f"    Incluye: Todos los registros con texto")
    
    # Archivo 3: Solo registros con texto válido (para análisis)
    archivo_texto = os.path.join(CARPETA_LIMPIA, 'publicaciones_texto.csv')
    df_texto_valido.to_csv(archivo_texto, index=False, encoding='utf-8-sig')
    print(f"\n  ✓ {archivo_texto}")
    print(f"    Total: {len(df_texto_valido)} registros")
    print(f"    Incluye: Solo texto válido (≥{LONGITUD_MINIMA_TEXTO} chars, ≥{PALABRAS_MINIMAS} palabras)")
    
    # Mostrar estadísticas generales
    print(f"\n{'='*80}")
    print("ESTADÍSTICAS GENERALES")
    print(f"{'='*80}")
    
    print(f"\nPor participante (registros válidos):")
    resumen = df_texto_valido.groupby('id_participante').agg({
        'id_publicacion': 'count',
        'longitud_limpia': 'mean',
        'num_palabras': 'mean'
    }).round(1)
    resumen.columns = ['total_publicaciones', 'promedio_caracteres', 'promedio_palabras']
    print(resumen.to_string())
    
    print(f"\nEstadísticas de texto limpio:")
    print(f"  - Promedio de caracteres: {df_texto_valido['longitud_limpia'].mean():.1f}")
    print(f"  - Promedio de palabras: {df_texto_valido['num_palabras'].mean():.1f}")
    print(f"  - Texto más corto: {df_texto_valido['longitud_limpia'].min():.0f} caracteres")
    print(f"  - Texto más largo: {df_texto_valido['longitud_limpia'].max():.0f} caracteres")
    
    print(f"\n{'='*80}")
    print("✓ PREPROCESAMIENTO COMPLETADO")
    print(f"{'='*80}")
    
    return df_texto_valido


In [ ]:
if __name__ == "__main__":
    df_resultado = preprocesar_datos()